In [1]:
#liberary
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from selenium import webdriver
import time
import datetime
import numpy as np
import glob
import os
import xlwings as xw

In [ ]:
#Put 1 file in the folder, only read first one 
tracker_path='D:\\06.SiteManager\\PO tracker'
file_list=glob.glob(tracker_path+'/*.xlsm')
PO_tracker=pd.read_excel(file_list[0],sheet_name='Report',skiprows=[0,2],header=[0],usecols='A,B,D,G,H,I,J,K,L,T,AO,AR,BC,BD,BE,BF,BG,BH,CC,CD,CE,CF,CG,CH,CI')
PO_tracker=PO_tracker.rename(columns = {'FB360 Commissioned':'Plan End Date','FB360 Commissioned.1':'Actual End Date'})

In [ ]:
#Seperate different type of site
Beacon_tracker=PO_tracker[(PO_tracker['Site Activity']=='Full Beacon') | (PO_tracker['Site Activity']=='2Step') | (PO_tracker['Site Activity']=='2Step Cold Build')]
Techadd_tracker=PO_tracker[(PO_tracker['Site Activity']=='L21') | (PO_tracker['Site Activity']=='L21 U9') | (PO_tracker['Site Activity']=='L26') | (PO_tracker['Site Activity']=='L21 10M') | (PO_tracker['Site Activity']=='TEF L18') | (PO_tracker['Site Activity']=='TEF L21') | (PO_tracker['Site Activity']=='TEF L26') | (PO_tracker['Site Activity']=='TEF Revisit') | (PO_tracker['Site Activity']=='5G')]

Creating the label column for sorting--Full Beacon

In [ ]:
#Classification of Beacon only or with 2step
def Classify(x):
    if '2Step'not in x:
        step_name=x+'2Step'
        if step_name in np.array(Beacon_tracker['DU ID']):
            return 'Full'
        else:
            return 'Beacon only'
    elif '2Step' in x:
        beacon_name=x.replace("2Step", "")
        if beacon_name in np.array(Beacon_tracker['DU ID']):
            return 'Attached'
        else:
            return '2Step only'
Beacon_tracker['Classification']=Beacon_tracker['DU ID'].apply(Classify)
#Beacon_tracker.groupby('Classification').size()

#Classification by Plan date
PO_date=2
cancel_point= pd.Timestamp.today() + pd.Timedelta(3, unit='M')
raise_point= pd.Timestamp.today() + pd.Timedelta(PO_date, unit='M')
today= pd.Timestamp.today()
def plan_status(row):
    if pd.notnull(row['Actual End Date']):
        return 'Finished'
    else:
        if pd.isnull(row['Plan End Date']):
            return 'de-instructed'
        elif row['Plan End Date'] > cancel_point:
            return 'Plan date postponed'
        elif ((row['Plan End Date'] <= raise_point)&(row['Plan End Date']>today)):
            return ('Process in '+str(PO_date)+' month')
        else:
            return 'Others'
Beacon_tracker['Plan_status'] = Beacon_tracker.apply (lambda row: plan_status(row),axis=1)    
#Beacon_tracker.groupby('Plan_status').size()

#Return related site to mysite column
def mysite(x):
    if '2Step'not in x:
        step_name=x+'2Step'
        if step_name in np.array(Beacon_tracker['DU ID']):
            return step_name
        else:
            return np.nan
    if '2Step' in x:
        beacon_name=x.replace("2Step", "")
        if beacon_name in np.array(Beacon_tracker['DU ID']):
            return beacon_name
        else:
            return np.nan
Beacon_tracker['Mysite']=Beacon_tracker['DU ID'].apply(mysite)
#replace site ID to the site status
Partner=Beacon_tracker.copy()
Partner.set_index('DU ID',inplace=True)
Beacon_tracker['Mysite']=Beacon_tracker['Mysite'].map(Partner['Plan_status'])
#Check PO status
Temp_active=Beacon_tracker['Active BTS I&C PO No.'].apply(lambda x: np.nan if 'Cancel' in str(x) else x)
Temp_passive=Beacon_tracker['Passive Partner Build PO'].apply(lambda x: np.nan if 'Cancel' in str(x) else x)
Need_2Step_PO=Beacon_tracker['Mysite'].apply(lambda x: True if x=='Finished'else False)
No_2Step_PO=pd.isnull(Beacon_tracker['Partner 2STEP to FB PO'])
Step_PO_noneed= ~(No_2Step_PO & Need_2Step_PO)
Beacon_tracker['PO_Covered']=((pd.notnull(Temp_active) | pd.notnull(Temp_passive)) & Step_PO_noneed)
#Beacon_tracker.groupby(['Plan_status','Classification','PO_Covered']).size()

Creating the label column for sorting--Techadd

In [ ]:
#Classification by Plan date
PO_date=2
cancel_point= pd.Timestamp.today() + pd.Timedelta(3, unit='M')
raise_point= pd.Timestamp.today() + pd.Timedelta(PO_date, unit='M')
today= pd.Timestamp.today() - pd.Timedelta(PO_date, unit='M') 
#today= pd.Timestamp(year=2019, month=1, day=1)
def plan_status(row):
    if pd.notnull(row['Actual End Date']):
        return 'Finished'
    else:
        if pd.isnull(row['Plan End Date']):
            return 'de-instructed'
        elif row['Plan End Date'] > cancel_point:
            return 'Plan date postponed'
        elif ((row['Plan End Date'] <= raise_point)&(row['Plan End Date']>=today)):
            return ('Process in '+str(PO_date)+' month')
        else:
            return 'Others'
Techadd_tracker['Plan_status'] = Techadd_tracker.apply (lambda row: plan_status(row),axis=1)    
#Techadd_tracker.groupby('Plan_status').size()

#Check PO status
Temp_Partner=Techadd_tracker['Partner Model PO'].apply(lambda x: np.nan if 'Cancel' in str(x) else x)
Techadd_tracker['PO_Covered']=pd.notnull(Temp_Partner)
#Techadd_tracker.groupby(['Plan_status','PO_Covered']).size()

Creating Cancel Table(Full information) & PO_to_cancel(For cancel program)--FullBeacon

In [ ]:
#Find Site needs to be cancelled, 1. not going 2. have PO 3.related site is not going ethier 
logic1=((Beacon_tracker['Plan_status']=='de-instructed') | (Beacon_tracker['Plan_status']=='Plan date postponed'))
logic2=Beacon_tracker['PO_Covered']
logic3=((Beacon_tracker['Mysite']=='de-instructed') | (Beacon_tracker['Mysite']=='Plan date postponed')| pd.isnull(Beacon_tracker['Mysite']))
Cancel_table=Beacon_tracker[logic1&logic2&logic3]
Cancel_table=Cancel_table.reset_index(drop=True)
#Cancel_table
#Final check, Attached should be same as Full
Cancel_table.groupby('Classification').size()
#Remove attached site(PO is not on attached site)
logic4=(Cancel_table['Classification']!='Attached')
Temp_cancel=Cancel_table[logic4].reset_index(drop=True)
#Active PO
Active_list=pd.DataFrame(columns=['PO NO.','Site Code','Reason'])
Active_list['PO NO.']=Temp_cancel['Active BTS I&C PO No.']
Active_list['Site Code']=Temp_cancel['DU ID']
Active_list['Reason']=Temp_cancel['Plan_status']
#Passive PO
Passive_list=pd.DataFrame(columns=['PO NO.','Site Code','Reason'])
Passive_list['PO NO.']=Temp_cancel['Passive Partner Build PO']
Passive_list['Site Code']=Temp_cancel['DU ID']
Passive_list['Reason']=Temp_cancel['Plan_status']
#Combine Table and organize data
PO_to_cancel=Active_list.append(Passive_list)
PO_to_cancel=PO_to_cancel.dropna()
PO_to_cancel['PO NO.']=PO_to_cancel['PO NO.'].apply(lambda x : x.strip('(old)').strip())
PO_to_cancel['PO NO.']=PO_to_cancel['PO NO.'].apply(lambda x : x.strip('(new)').strip())
PO_to_cancel=PO_to_cancel.drop_duplicates()
PO_to_cancel=PO_to_cancel.sort_values('Reason').reset_index(drop=True)

Creating FulBeacon PO table

In [ ]:
logic5=(Beacon_tracker['Plan_status']==('Process in '+str(PO_date)+' month'))
PO_Summary_FB=Beacon_tracker[logic5]
logic6=(Techadd_tracker['Plan_status']==('Process in '+str(PO_date)+' month'))
PO_Summary_Techadd=Techadd_tracker[logic6]

Prepare E2E PO table

In [ ]:
def Concract_fixed(x):
    try:
        start=x.find('0008')
        return x[start:(start+14)]
    except:
        return x
PO_E2E=PO_Summary_FB[(PO_Summary_FB['PO_Covered']==False)&(pd.isnull(PO_Summary_FB['Active BTS I&C PO No.']))]
PO_E2E=PO_E2E[['DU ID','Partner','Build Type','Site Type','Active VOD I&C Contract number','Passive Huawei Contract Number','QS Model']]
PO_E2E['Active VOD I&C Contract number']=PO_E2E['Active VOD I&C Contract number'].apply(lambda x :Concract_fixed(str(x)))
PO_E2E['Passive Huawei Contract Number']=PO_E2E['Passive Huawei Contract Number'].apply(lambda x :Concract_fixed(str(x)))
PO_E2E=PO_E2E.reset_index()

Prepare Revisit PO table

In [ ]:
PO_Revisit=PO_Summary_FB[(PO_Summary_FB['PO_Covered']==False)&(pd.isnull(PO_Summary_FB['Partner 2STEP to FB PO']))&(PO_Summary_FB['Mysite']=='Finished')]
PO_Revisit=PO_Revisit[['DU ID','Partner','FB Revisit HW contract No.']]
PO_Revisit['FB Revisit HW contract No.']=PO_Revisit['FB Revisit HW contract No.'].apply(lambda x :Concract_fixed(str(x)))
PO_Revisit=PO_Revisit.reset_index()

Prepare Techadd PO Table

In [ ]:
PO_Techadd=PO_Summary_Techadd[(PO_Summary_Techadd['PO_Covered']==False)&(pd.isnull(PO_Summary_Techadd['Partner Model PO']))]
PO_Techadd=PO_Techadd[['DU ID','Partner','SPIM 2 VF PO Received','Partner Model']]
PO_Techadd['SPIM 2 VF PO Received']=PO_Techadd['SPIM 2 VF PO Received'].apply(lambda x :Concract_fixed(str(x)))
PO_Techadd=PO_Techadd.reset_index()

Writing in EXCEL

In [ ]:
Today=datetime.date.today()
Filename_with_route=r'D:\\06.SiteManager\\SiteManager_'+Today.strftime('%d%m%Y')+'.xlsx'
writer = pd.ExcelWriter(Filename_with_route)
PO_Summary_FB.to_excel(writer,'FB_Summary')
PO_Summary_Techadd.to_excel(writer,'Techadd_Summary')
PO_E2E.to_excel(writer,"E2E_PO_ready")
PO_Revisit.to_excel(writer,"Revisit_PO_ready")
PO_Techadd.to_excel(writer,"Techadd_PO_ready")
Cancel_table.to_excel(writer,'E2E_Cancel_Summary')
PO_to_cancel.to_excel(writer,'sheet1')

writer.save()

PR_template V1.0

In [ ]:
#item Database
item_base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\FB_PR_item.xlsx'
AGF = pd.read_excel(item_base, sheet_name='AGF',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
ART = pd.read_excel(item_base, sheet_name='ART',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
AST = pd.read_excel(item_base, sheet_name='AST',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
RT = pd.read_excel(item_base, sheet_name='RT',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
ST = pd.read_excel(item_base, sheet_name='ST',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM1 = pd.read_excel(item_base, sheet_name='GFM1',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM2 = pd.read_excel(item_base, sheet_name='GFM2',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM3 = pd.read_excel(item_base, sheet_name='GFM3',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM4 = pd.read_excel(item_base, sheet_name='GFM4',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM5 = pd.read_excel(item_base, sheet_name='GFM5',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
GFM6 = pd.read_excel(item_base, sheet_name='GFM6',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
MWR = pd.read_excel(item_base, sheet_name='MWR',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
RE2 = pd.read_excel(item_base, sheet_name='RE2',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
REWR = pd.read_excel(item_base, sheet_name='REWR',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')

Normal reading when running whole program

In [ ]:
PO_ready=pd.read_excel(Filename_with_route,sheet_name="E2E_PO_ready",dtype=str)

Reading file without setting route

In [ ]:
PO_ready=pd.read_excel(Filename_with_route,sheet_name="E2E_PO_ready",dtype=str)

Main E2E PR

In [ ]:
#Main E2E PO
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
PR_ALL=pd.read_excel(Base,sheet_name='PR ENG',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')

for row in range(0,len(PO_ready)):
    if PO_ready.loc[row, 'Build Type']=='Active only':
        if PO_ready.loc[row, 'Site Type']=='Streetworks':
            Temp=AST.copy()
            Temp['CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
            Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
            Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
            PR_ALL=PR_ALL.append(Temp, ignore_index=False)     
        elif PO_ready.loc[row, 'Site Type']=='Rooftop':
            Temp=ART.copy()
            Temp['CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
            Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
            Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
            PR_ALL=PR_ALL.append(Temp, ignore_index=False)
        elif PO_ready.loc[row, 'Site Type']=='Greenfield':
            Temp=AGF.copy()
            Temp['CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
            Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
            Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
            PR_ALL=PR_ALL.append(Temp, ignore_index=False)
        else:
            print("Something is incorrect, please check")
    else:
        if PO_ready.loc[row, 'Site Type']=='Streetworks':
            Temp=ST.copy()
            Temp.loc[[0,1],'CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
            Temp.loc[2,'CONTRACT_NO']=PO_ready.loc[row, 'Passive Huawei Contract Number']
            Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
            Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
            PR_ALL=PR_ALL.append(Temp, ignore_index=False)
        elif PO_ready.loc[row, 'Site Type']=='Rooftop':
            Temp=RT.copy()
            Temp.loc[[0,1],'CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
            Temp.loc[2,'CONTRACT_NO']=PO_ready.loc[row, 'Passive Huawei Contract Number']
            Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
            Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
            PR_ALL=PR_ALL.append(Temp, ignore_index=False)
        elif PO_ready.loc[row, 'Site Type']=='Greenfield':
            if PO_ready.loc[row, 'QS Model']=='M1':
                Temp=GFM1.copy()
                Temp.loc[[0,1],'CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
                Temp.loc[2,'CONTRACT_NO']=PO_ready.loc[row, 'Passive Huawei Contract Number']
                Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
                Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
                PR_ALL=PR_ALL.append(Temp, ignore_index=False)
            elif PO_ready.loc[row, 'QS Model']=='M2':
                Temp=GFM2.copy()
                Temp.loc[[0,1],'CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
                Temp.loc[2,'CONTRACT_NO']=PO_ready.loc[row, 'Passive Huawei Contract Number']
                Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
                Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
                PR_ALL=PR_ALL.append(Temp, ignore_index=False)
            elif PO_ready.loc[row, 'QS Model']=='M3':
                Temp=GFM3.copy()
                Temp.loc[[0,1],'CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
                Temp.loc[2,'CONTRACT_NO']=PO_ready.loc[row, 'Passive Huawei Contract Number']
                Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
                Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
                PR_ALL=PR_ALL.append(Temp, ignore_index=False)
            elif PO_ready.loc[row, 'QS Model']=='M4':
                Temp=GFM4.copy()
                Temp.loc[[0,1],'CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
                Temp.loc[2,'CONTRACT_NO']=PO_ready.loc[row, 'Passive Huawei Contract Number']
                Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
                Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
                PR_ALL=PR_ALL.append(Temp, ignore_index=False)
            elif PO_ready.loc[row, 'QS Model']=='M5':
                Temp=GFM5.copy()
                Temp.loc[[0,1],'CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
                Temp.loc[2,'CONTRACT_NO']=PO_ready.loc[row, 'Passive Huawei Contract Number']
                Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
                Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
                PR_ALL=PR_ALL.append(Temp, ignore_index=False)
            elif PO_ready.loc[row, 'QS Model']=='M6':
                Temp=GFM6.copy()
                Temp.loc[[0,1],'CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
                Temp.loc[2,'CONTRACT_NO']=PO_ready.loc[row, 'Passive Huawei Contract Number']
                Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
                Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
                PR_ALL=PR_ALL.append(Temp, ignore_index=False)
            else:
                print("Something is incorrect, please check") 
        else:
            print("Something is incorrect, please check")       

In [ ]:
#Fixed time format
Start=pd.Timestamp.today() + pd.Timedelta(7, unit='D')
End=pd.Timestamp.today() + pd.Timedelta(21, unit='D')
PR_ALL['START_DATE']=Start.strftime('%Y-%m-%d')
PR_ALL['END_DATE']=End.strftime('%Y-%m-%d')

In [ ]:
#Writing formal template
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
app=xw.App(visible=True,add_book=False)
wb=app.books.open(Base)
sht = wb.sheets['PR ENG']
sht.range('A4').value=np.vstack(np.array(PR_ALL['REGION_CODE']))
sht.range('C4').value=np.vstack(np.array(PR_ALL['PROJECT_CODE']))
sht.range('D4').value=np.vstack(np.array(PR_ALL['SUB_PROJECT_CODE']))
sht.range('E4').value=np.vstack(np.array(PR_ALL['CONTRACT_NO']))
sht.range('G4').value=np.vstack(np.array(PR_ALL['ENGINEERING_NAME']))
sht.range('J4').value=np.vstack(np.array(PR_ALL['REQUESTOR_EMPLOYEE_NUM']))
sht.range('N4').value=np.vstack(np.array(PR_ALL['DEPARTMENT_CODE']))
sht.range('O4').value=np.vstack(np.array(PR_ALL['DU_CODE']))
sht.range('Q4').value=np.vstack(np.array(PR_ALL['ITEM_CODE']))
sht.range('R4').value=np.vstack(np.array(PR_ALL['ITEM_DESC']))
sht.range('V4').value=np.vstack(np.array(PR_ALL['UOM']))
sht.range('W4').value=np.vstack(np.array(PR_ALL['QTY']))
sht.range('X4').value=np.vstack(np.array(PR_ALL['UNIT_PRICE']))
sht.range('Y4').value=np.vstack(np.array(PR_ALL['CURRENCY']))
sht.range('Z4').value=np.vstack(np.array(PR_ALL['SERVICE_PRODUCT_CODE']))
sht.range('AA4').value=np.vstack(np.array(PR_ALL['START_DATE']))
sht.range('AB4').value=np.vstack(np.array(PR_ALL['END_DATE']))

War room PR

In [ ]:
#War room
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
PR_ALL=pd.read_excel(Base,sheet_name='PR ENG',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
for row in range(0,len(PO_ready)):
    Temp=MWR.copy()
    Temp['CONTRACT_NO']=PO_ready.loc[row, 'Active VOD I&C Contract number']
    #Temp['ENGINEERING_NAME']=PO_ready.loc[row, 'Partner']
    Temp['DU_CODE']=PO_ready.loc[row, 'DU ID']
    PR_ALL=PR_ALL.append(Temp, ignore_index=False) 

In [ ]:
#Fixed time format
Start=pd.Timestamp.today() + pd.Timedelta(7, unit='D')
End=pd.Timestamp.today() + pd.Timedelta(21, unit='D')
PR_ALL['START_DATE']=Start.strftime('%Y-%m-%d')
PR_ALL['END_DATE']=End.strftime('%Y-%m-%d')

In [ ]:
#Writing formal template
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
app=xw.App(visible=True,add_book=False)
wb=app.books.open(Base)
sht = wb.sheets['PR ENG']
sht.range('A4').value=np.vstack(np.array(PR_ALL['REGION_CODE']))
sht.range('C4').value=np.vstack(np.array(PR_ALL['PROJECT_CODE']))
sht.range('D4').value=np.vstack(np.array(PR_ALL['SUB_PROJECT_CODE']))
sht.range('E4').value=np.vstack(np.array(PR_ALL['CONTRACT_NO']))
sht.range('G4').value=np.vstack(np.array(PR_ALL['ENGINEERING_NAME']))
sht.range('J4').value=np.vstack(np.array(PR_ALL['REQUESTOR_EMPLOYEE_NUM']))
sht.range('N4').value=np.vstack(np.array(PR_ALL['DEPARTMENT_CODE']))
sht.range('O4').value=np.vstack(np.array(PR_ALL['DU_CODE']))
sht.range('Q4').value=np.vstack(np.array(PR_ALL['ITEM_CODE']))
sht.range('R4').value=np.vstack(np.array(PR_ALL['ITEM_DESC']))
sht.range('V4').value=np.vstack(np.array(PR_ALL['UOM']))
sht.range('W4').value=np.vstack(np.array(PR_ALL['QTY']))
sht.range('X4').value=np.vstack(np.array(PR_ALL['UNIT_PRICE']))
sht.range('Y4').value=np.vstack(np.array(PR_ALL['CURRENCY']))
sht.range('Z4').value=np.vstack(np.array(PR_ALL['SERVICE_PRODUCT_CODE']))
sht.range('AA4').value=np.vstack(np.array(PR_ALL['START_DATE']))
sht.range('AB4').value=np.vstack(np.array(PR_ALL['END_DATE']))

2Step to Full Beacon Revisit PR

In [ ]:
PO_Revisit=pd.read_excel(Filename_with_route,sheet_name="Revisit_PO_ready",dtype=str)

In [ ]:
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
PR_ALL=pd.read_excel(Base,sheet_name='PR ENG',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
for row in range(0,len(PO_Revisit)):
    Temp=RE2.copy()
    Temp['CONTRACT_NO']=PO_Revisit.loc[row, 'FB Revisit HW contract No.']
    Temp['ENGINEERING_NAME']=PO_Revisit.loc[row, 'Partner']
    Temp['DU_CODE']=PO_Revisit.loc[row, 'DU ID']
    PR_ALL=PR_ALL.append(Temp, ignore_index=False) 

2Step to Full Beacon Revisit War room PR

In [ ]:
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
PR_ALL=pd.read_excel(Base,sheet_name='PR ENG',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
for row in range(0,len(PO_Revisit)):
    Temp=REWR.copy()
    Temp['CONTRACT_NO']=PO_Revisit.loc[row, 'FB Revisit HW contract No.']
    #Temp['ENGINEERING_NAME']=PO_Revisit.loc[row, 'Partner']
    Temp['DU_CODE']=PO_Revisit.loc[row, 'DU ID']
    PR_ALL=PR_ALL.append(Temp, ignore_index=False) 

In [ ]:
#PR_file_name="PR"+now.strftime("%H%M")+str(SN)+"_"+sub_con_name+"_"+str(datetime.date.today())+'.xlsx'
#Whole_route='D:\\03.Purchase Reqeust\\02.Process\\'+PR_file_name
#wb.save(Whole_route)
#wb.close()
#app.quit()

Tech Add

In [2]:
#item Database
item_base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\Techadd_PR_item.xlsx'
TAM0 = pd.read_excel(item_base, sheet_name='M0',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TAM1 = pd.read_excel(item_base, sheet_name='M1',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TAM2 = pd.read_excel(item_base, sheet_name='M2',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TAM3 = pd.read_excel(item_base, sheet_name='M3',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TAM4 = pd.read_excel(item_base, sheet_name='M4',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TAM5 = pd.read_excel(item_base, sheet_name='M5',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TAM6 = pd.read_excel(item_base, sheet_name='M6',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TAM7 = pd.read_excel(item_base, sheet_name='M7',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TAWR = pd.read_excel(item_base, sheet_name='OMC',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TADD = pd.read_excel(item_base, sheet_name='Design',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TASV = pd.read_excel(item_base, sheet_name='Survey',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TALT = pd.read_excel(item_base, sheet_name='Logistic',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TAG51ST = pd.read_excel(item_base, sheet_name='G51STV',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TAG52ND = pd.read_excel(item_base, sheet_name='G52NDV',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
G5TRIAL = pd.read_excel(item_base, sheet_name='G5TRIAL',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')

Manual Input

In [3]:
Tool='D:\\03.Purchase Reqeust\\Techadd_template_writing.xlsx'
PO_tech=pd.read_excel(Tool,sheet_name='PO_laucher',dtype=str)
PO_tech

,DU ID,Partner,SPIM 2 VF PO Received,Partner Model,VDF PO
0,10477O2TDD,Virtua,00082619201822,Survey,nan
1,36407O2TDD,Virtua,00082619202722,Survey,nan
2,1990O2TDD,Virtua,00082619202322,Survey,nan


Tracker Input

In [ ]:
PO_tech=pd.read_excel(Filename_with_route,sheet_name='Techadd_PO_ready',dtype=str)
PO_tech=PO_tech.reset_index(drop=False)
PO_tech

In [4]:
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
PR_ALL=pd.read_excel(Base,sheet_name='PR ENG',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
for row in range(0,len(PO_tech)):
    if PO_tech.loc[row, 'Partner Model']=='Model 0':
        Temp=TAM0.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_tech.loc[row, 'Partner Model']=='Model 1':
        Temp=TAM1.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_tech.loc[row, 'Partner Model']=='Model 2':
        Temp=TAM2.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_tech.loc[row, 'Partner Model']=='Model 3':
        Temp=TAM3.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_tech.loc[row, 'Partner Model']=='Model 4':
        Temp=TAM4.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_tech.loc[row, 'Partner Model']=='Model 5':
        Temp=TAM5.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_tech.loc[row, 'Partner Model']=='Model 6':
        Temp=TAM6.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_tech.loc[row, 'Partner Model']=='Model 7':
        Temp=TAM7.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    elif PO_tech.loc[row, 'Partner Model']=='Design':
        Temp=TADD.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    elif PO_tech.loc[row, 'Partner Model']=='Survey':
        Temp=TASV.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    elif PO_tech.loc[row, 'Partner Model']=='First Visit':
        Temp=TAG51ST.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    elif PO_tech.loc[row, 'Partner Model']=='Second Visit':
        Temp=TAG52ND.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    elif PO_tech.loc[row, 'Partner Model']=='Logistic':
        Temp=TALT.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    elif PO_tech.loc[row, 'Partner Model']=='G5TRIAL':
        Temp=G5TRIAL.copy()
        Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
        Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    else:
        print("Something is incorrect, please check")       

In [5]:
#Fixed time format
Start=pd.Timestamp.today() + pd.Timedelta(7, unit='D')
End=pd.Timestamp.today() + pd.Timedelta(21, unit='D')
PR_ALL['START_DATE']=Start.strftime('%Y-%m-%d')
PR_ALL['END_DATE']=End.strftime('%Y-%m-%d')

In [8]:
#Writing formal template
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
app=xw.App(visible=True,add_book=False)
wb=app.books.open(Base)
sht = wb.sheets['PR ENG']
sht.range('A4').value=np.vstack(np.array(PR_ALL['REGION_CODE']))
sht.range('C4').value=np.vstack(np.array(PR_ALL['PROJECT_CODE']))
sht.range('D4').value=np.vstack(np.array(PR_ALL['SUB_PROJECT_CODE']))
sht.range('E4').value=np.vstack(np.array(PR_ALL['CONTRACT_NO']))
sht.range('G4').value=np.vstack(np.array(PR_ALL['ENGINEERING_NAME']))
sht.range('J4').value=np.vstack(np.array(PR_ALL['REQUESTOR_EMPLOYEE_NUM']))
sht.range('N4').value=np.vstack(np.array(PR_ALL['DEPARTMENT_CODE']))
sht.range('O4').value=np.vstack(np.array(PR_ALL['DU_CODE']))
sht.range('Q4').value=np.vstack(np.array(PR_ALL['ITEM_CODE']))
sht.range('R4').value=np.vstack(np.array(PR_ALL['ITEM_DESC']))
sht.range('V4').value=np.vstack(np.array(PR_ALL['UOM']))
sht.range('W4').value=np.vstack(np.array(PR_ALL['QTY']))
sht.range('X4').value=np.vstack(np.array(PR_ALL['UNIT_PRICE']))
sht.range('Y4').value=np.vstack(np.array(PR_ALL['CURRENCY']))
sht.range('Z4').value=np.vstack(np.array(PR_ALL['SERVICE_PRODUCT_CODE']))
sht.range('AA4').value=np.vstack(np.array(PR_ALL['START_DATE']))
sht.range('AB4').value=np.vstack(np.array(PR_ALL['END_DATE']))

Tech add War room PR 

In [ ]:
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
PR_ALL=pd.read_excel(Base,sheet_name='PR ENG',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
for row in range(0,len(PO_tech)):
    Temp=TAWR.copy()
    Temp['CONTRACT_NO']=PO_tech.loc[row, 'SPIM 2 VF PO Received']
    #Temp['ENGINEERING_NAME']=PO_tech.loc[row, 'Partner']
    Temp['DU_CODE']=PO_tech.loc[row, 'DU ID']
    PR_ALL=PR_ALL.append(Temp, ignore_index=False)  

In [ ]:
#Fixed time format
Start=pd.Timestamp.today() + pd.Timedelta(7, unit='D')
End=pd.Timestamp.today() + pd.Timedelta(21, unit='D')
PR_ALL['START_DATE']=Start.strftime('%Y-%m-%d')
PR_ALL['END_DATE']=End.strftime('%Y-%m-%d')

In [ ]:
#Writing formal template
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
app=xw.App(visible=True,add_book=False)
wb=app.books.open(Base)
sht = wb.sheets['PR ENG']
sht.range('A4').value=np.vstack(np.array(PR_ALL['REGION_CODE']))
sht.range('C4').value=np.vstack(np.array(PR_ALL['PROJECT_CODE']))
sht.range('D4').value=np.vstack(np.array(PR_ALL['SUB_PROJECT_CODE']))
sht.range('E4').value=np.vstack(np.array(PR_ALL['CONTRACT_NO']))
sht.range('G4').value=np.vstack(np.array(PR_ALL['ENGINEERING_NAME']))
sht.range('J4').value=np.vstack(np.array(PR_ALL['REQUESTOR_EMPLOYEE_NUM']))
sht.range('N4').value=np.vstack(np.array(PR_ALL['DEPARTMENT_CODE']))
sht.range('O4').value=np.vstack(np.array(PR_ALL['DU_CODE']))
sht.range('Q4').value=np.vstack(np.array(PR_ALL['ITEM_CODE']))
sht.range('R4').value=np.vstack(np.array(PR_ALL['ITEM_DESC']))
sht.range('V4').value=np.vstack(np.array(PR_ALL['UOM']))
sht.range('W4').value=np.vstack(np.array(PR_ALL['QTY']))
sht.range('X4').value=np.vstack(np.array(PR_ALL['UNIT_PRICE']))
sht.range('Y4').value=np.vstack(np.array(PR_ALL['CURRENCY']))
sht.range('Z4').value=np.vstack(np.array(PR_ALL['SERVICE_PRODUCT_CODE']))
sht.range('AA4').value=np.vstack(np.array(PR_ALL['START_DATE']))
sht.range('AB4').value=np.vstack(np.array(PR_ALL['END_DATE']))

Connnect44 Signal Testing PR

In [ ]:
#item Database
item_base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\C44_PR_item.xlsx'
TAC44 = pd.read_excel(item_base, sheet_name='TECHADD',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
G5C44 = pd.read_excel(item_base, sheet_name='5G',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
MIMOC44 = pd.read_excel(item_base, sheet_name='MaMIMO',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')

In [ ]:
Tool='D:\\03.Purchase Reqeust\\Connect44_spare_template_writing.xlsx'
PO_C44=pd.read_excel(Tool,sheet_name='PO_laucher',dtype=str)
PO_C44

In [ ]:
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
PR_ALL=pd.read_excel(Base,sheet_name='PR ENG',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
for row in range(0,len(PO_C44)):
    if PO_C44.loc[row, 'Partner Model']=='TECHADD':
        Temp=TAC44.copy()
        Temp['CONTRACT_NO']=PO_C44.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_C44.loc[row, 'Partner']
        Temp['DU_CODE']=PO_C44.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_C44.loc[row, 'Partner Model']=='5G':
        Temp=G5C44.copy()
        Temp['CONTRACT_NO']=PO_C44.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_C44.loc[row, 'Partner']
        Temp['DU_CODE']=PO_C44.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_C44.loc[row, 'Partner Model']=='MAMIMO':
        Temp=MIMOC44.copy()
        Temp['CONTRACT_NO']=PO_C44.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_C44.loc[row, 'Partner']
        Temp['DU_CODE']=PO_C44.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    else:
        print("Something is incorrect, please check")  

In [ ]:
#Fixed time format
Start=pd.Timestamp.today() + pd.Timedelta(7, unit='D')
End=pd.Timestamp.today() + pd.Timedelta(21, unit='D')
PR_ALL['START_DATE']=Start.strftime('%Y-%m-%d')
PR_ALL['END_DATE']=End.strftime('%Y-%m-%d')

In [ ]:
#Writing formal template
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
app=xw.App(visible=True,add_book=False)
wb=app.books.open(Base)
sht = wb.sheets['PR ENG']
sht.range('A4').value=np.vstack(np.array(PR_ALL['REGION_CODE']))
sht.range('C4').value=np.vstack(np.array(PR_ALL['PROJECT_CODE']))
sht.range('D4').value=np.vstack(np.array(PR_ALL['SUB_PROJECT_CODE']))
sht.range('E4').value=np.vstack(np.array(PR_ALL['CONTRACT_NO']))
sht.range('G4').value=np.vstack(np.array(PR_ALL['ENGINEERING_NAME']))
sht.range('J4').value=np.vstack(np.array(PR_ALL['REQUESTOR_EMPLOYEE_NUM']))
sht.range('N4').value=np.vstack(np.array(PR_ALL['DEPARTMENT_CODE']))
sht.range('O4').value=np.vstack(np.array(PR_ALL['DU_CODE']))
sht.range('Q4').value=np.vstack(np.array(PR_ALL['ITEM_CODE']))
sht.range('R4').value=np.vstack(np.array(PR_ALL['ITEM_DESC']))
sht.range('V4').value=np.vstack(np.array(PR_ALL['UOM']))
sht.range('W4').value=np.vstack(np.array(PR_ALL['QTY']))
sht.range('X4').value=np.vstack(np.array(PR_ALL['UNIT_PRICE']))
sht.range('Y4').value=np.vstack(np.array(PR_ALL['CURRENCY']))
sht.range('Z4').value=np.vstack(np.array(PR_ALL['SERVICE_PRODUCT_CODE']))
sht.range('AA4').value=np.vstack(np.array(PR_ALL['START_DATE']))
sht.range('AB4').value=np.vstack(np.array(PR_ALL['END_DATE']))

TDD Design PO

In [6]:
#item Database
item_base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\TDD_Design_PR_item.xlsx'
TDDGF = pd.read_excel(item_base, sheet_name='GF',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TDDRT = pd.read_excel(item_base, sheet_name='RT',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
TDDSV = pd.read_excel(item_base, sheet_name='Survey',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')

In [7]:
Tool='D:\\03.Purchase Reqeust\\TDD_Design_spare_template_writing.xlsx'
PO_TDD=pd.read_excel(Tool,sheet_name='PO_laucher',dtype=str)
PO_TDD

,DU ID,Partner,VDF PO,SPIM 2 VF PO Received,Partner Model
0,1670VFTDD,Sitec,nan,00082618211323,RT
1,1670VFTDD,Sitec,nan,00082618211323,Survey


In [8]:
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
PR_ALL=pd.read_excel(Base,sheet_name='PR ENG',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
for row in range(0,len(PO_TDD)):
    if PO_TDD.loc[row, 'Partner Model']=='RT':
        Temp=TDDRT.copy()
        Temp['CONTRACT_NO']=PO_TDD.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_TDD.loc[row, 'Partner']
        Temp['DU_CODE']=PO_TDD.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_TDD.loc[row, 'Partner Model']=='GF':
        Temp=TDDGF.copy()
        Temp['CONTRACT_NO']=PO_TDD.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_TDD.loc[row, 'Partner']
        Temp['DU_CODE']=PO_TDD.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    elif PO_TDD.loc[row, 'Partner Model']=='Survey':
        Temp=TDDSV.copy()
        Temp['CONTRACT_NO']=PO_TDD.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_TDD.loc[row, 'Partner']
        Temp['DU_CODE']=PO_TDD.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    else:
        print("Something is incorrect, please check")  

In [9]:
#Fixed time format
Start=pd.Timestamp.today() + pd.Timedelta(7, unit='D')
End=pd.Timestamp.today() + pd.Timedelta(21, unit='D')
PR_ALL['START_DATE']=Start.strftime('%Y-%m-%d')
PR_ALL['END_DATE']=End.strftime('%Y-%m-%d')

In [10]:
#Writing formal template
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
app=xw.App(visible=True,add_book=False)
wb=app.books.open(Base)
sht = wb.sheets['PR ENG']
sht.range('A4').value=np.vstack(np.array(PR_ALL['REGION_CODE']))
sht.range('C4').value=np.vstack(np.array(PR_ALL['PROJECT_CODE']))
sht.range('D4').value=np.vstack(np.array(PR_ALL['SUB_PROJECT_CODE']))
sht.range('E4').value=np.vstack(np.array(PR_ALL['CONTRACT_NO']))
sht.range('G4').value=np.vstack(np.array(PR_ALL['ENGINEERING_NAME']))
sht.range('J4').value=np.vstack(np.array(PR_ALL['REQUESTOR_EMPLOYEE_NUM']))
sht.range('N4').value=np.vstack(np.array(PR_ALL['DEPARTMENT_CODE']))
sht.range('O4').value=np.vstack(np.array(PR_ALL['DU_CODE']))
sht.range('Q4').value=np.vstack(np.array(PR_ALL['ITEM_CODE']))
sht.range('R4').value=np.vstack(np.array(PR_ALL['ITEM_DESC']))
sht.range('V4').value=np.vstack(np.array(PR_ALL['UOM']))
sht.range('W4').value=np.vstack(np.array(PR_ALL['QTY']))
sht.range('X4').value=np.vstack(np.array(PR_ALL['UNIT_PRICE']))
sht.range('Y4').value=np.vstack(np.array(PR_ALL['CURRENCY']))
sht.range('Z4').value=np.vstack(np.array(PR_ALL['SERVICE_PRODUCT_CODE']))
sht.range('AA4').value=np.vstack(np.array(PR_ALL['START_DATE']))
sht.range('AB4').value=np.vstack(np.array(PR_ALL['END_DATE']))

QEHS PR

In [ ]:
#item Database
item_base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\QEHS_PR_item.xlsx'
Street = pd.read_excel(item_base, sheet_name='Street',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
SSow = pd.read_excel(item_base, sheet_name='SSow',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
QA = pd.read_excel(item_base, sheet_name='QA',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')

In [ ]:
Tool='D:\\03.Purchase Reqeust\\QEHS_template_writing.xlsx'
PO_QEHS=pd.read_excel(Tool,sheet_name='PO_laucher',dtype=str)
PO_QEHS

In [ ]:
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
PR_ALL=pd.read_excel(Base,sheet_name='PR ENG',skiprows=[1,2],dtype=str).replace('nan', np.nan).dropna(how='all')
for row in range(0,len(PO_QEHS)):
    if PO_QEHS.loc[row, 'Partner Model']=='Street':
        Temp=Street.copy()
        Temp['CONTRACT_NO']=PO_QEHS.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_TDD.loc[row, 'Partner']
        Temp['DU_CODE']=PO_TDD.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)  
    elif PO_QEHS.loc[row, 'Partner Model']=='SSow':
        Temp=SSow.copy()
        Temp['CONTRACT_NO']=PO_TDD.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_TDD.loc[row, 'Partner']
        Temp['DU_CODE']=PO_TDD.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    elif PO_QEHS.loc[row, 'Partner Model']=='QA':
        Temp=QA.copy()
        Temp['CONTRACT_NO']=PO_TDD.loc[row, 'SPIM 2 VF PO Received']
        Temp['ENGINEERING_NAME']=PO_TDD.loc[row, 'Partner']
        Temp['DU_CODE']=PO_TDD.loc[row, 'DU ID']
        PR_ALL=PR_ALL.append(Temp, ignore_index=False)
    else:
        print("Something is incorrect, please check")  

In [ ]:
#Fixed time format
Start=pd.Timestamp.today() + pd.Timedelta(7, unit='D')
End=pd.Timestamp.today() + pd.Timedelta(21, unit='D')
PR_ALL['START_DATE']=Start.strftime('%Y-%m-%d')
PR_ALL['END_DATE']=End.strftime('%Y-%m-%d')

In [ ]:
#Writing formal template
Base=r'D:\\03.Purchase Reqeust\\Database_and_Configuration\\PR_ebuy_template_empty.xlsx'
app=xw.App(visible=True,add_book=False)
wb=app.books.open(Base)
sht = wb.sheets['PR ENG']
sht.range('A4').value=np.vstack(np.array(PR_ALL['REGION_CODE']))
sht.range('C4').value=np.vstack(np.array(PR_ALL['PROJECT_CODE']))
sht.range('D4').value=np.vstack(np.array(PR_ALL['SUB_PROJECT_CODE']))
sht.range('E4').value=np.vstack(np.array(PR_ALL['CONTRACT_NO']))
sht.range('G4').value=np.vstack(np.array(PR_ALL['ENGINEERING_NAME']))
sht.range('J4').value=np.vstack(np.array(PR_ALL['REQUESTOR_EMPLOYEE_NUM']))
sht.range('N4').value=np.vstack(np.array(PR_ALL['DEPARTMENT_CODE']))
sht.range('O4').value=np.vstack(np.array(PR_ALL['DU_CODE']))
sht.range('Q4').value=np.vstack(np.array(PR_ALL['ITEM_CODE']))
sht.range('R4').value=np.vstack(np.array(PR_ALL['ITEM_DESC']))
sht.range('V4').value=np.vstack(np.array(PR_ALL['UOM']))
sht.range('W4').value=np.vstack(np.array(PR_ALL['QTY']))
sht.range('X4').value=np.vstack(np.array(PR_ALL['UNIT_PRICE']))
sht.range('Y4').value=np.vstack(np.array(PR_ALL['CURRENCY']))
sht.range('Z4').value=np.vstack(np.array(PR_ALL['SERVICE_PRODUCT_CODE']))
sht.range('AA4').value=np.vstack(np.array(PR_ALL['START_DATE']))
sht.range('AB4').value=np.vstack(np.array(PR_ALL['END_DATE']))